In [1]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

In [2]:
# 1. Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles
    
    return movie

In [63]:
# 2 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def extract_transform_load(wiki_file, kaggle_file, ratings_file):
    # Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_metadata = pd.read_csv('movies_metadata.csv', low_memory=False)
    ratings = pd.read_csv('ratings.csv')

    # Open and read the Wikipedia data JSON file.
    #wiki_movies_df = pd.read_json('wikipedia-movies.json')                           
    with open('wikipedia-movies.json', mode='r') as file:
        wiki_movies_raw = json.load(file)
        wiki_movies_df = pd.DataFrame(wiki_movies_raw)
        
    
    # 3. Write a list comprehension to filter out TV shows.
    wiki_movies = [movie for movie in wiki_movies_raw
        if('Television series' in movie and 'No. of episodes' not in movie)]
    #print(wiki_movies_df.columns)
    # 4. Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    
    clean_movies = [clean_movie(movie) for movie in wiki_movies]

    # 5. Read in the cleaned movies list from Step 4 as a DataFrame.
                   
    wiki_movies = pd.DataFrame(clean_movies)
    
    # 6. Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    try:
        wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
        wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
    except Exception as e:
        print(e)

    #  7. Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
    wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]
    wiki_movies_df

    # 8. Create a variable that will hold the non-null values from the “Box office” column.
    #[[column,wiki_movies_df[column].notnull().sum()] for column in wiki_movies_df.columns]
   
    box_office = wiki_movies_df['Box office'].dropna()
    
    # 9. Convert the box office data created in Step 8 to string values using the lambda and join functions.
    box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)

    # 10. Write a regular expression to match the six elements of "form_one" of the box office data.
    form_one = r'\$\d+\.?\d*\s*[mb]illion'
    # 11. Write a regular expression to match the three elements of "form_two" of the box office data.
    form_two = r'\$\d{1,3}(?:,\d{3})+'
    
    budget = wiki_movies_df['Budget'].dropna()
    
    release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    
    date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]\d,\s\d{4}'
    date_form_two = r'\d{4}.[01]\d.[123]\d'
    date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
    date_form_four = r'\d{4}'
    
    running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    
    running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')
    running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)
    
    # 12. Add the parse_dollars function.
    def parse_dollars(s):
        # if s is not a string, return NaN
        if type(s) != str:
            return np.nan

        # if input is of the form $###.# million
        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

            # remove dollar sign and " million"
            s = re.sub('\$|\s|[a-zA-Z]','', s)

            # convert to float and multiply by a million
            value = float(s) * 10**6

            # return value
            return value

        # if input is of the form $###.# billion
        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

            # remove dollar sign and " billion"
            s = re.sub('\$|\s|[a-zA-Z]','', s)

            # convert to float and multiply by a billion
            value = float(s) * 10**9

            # return value
            return value

        # if input is of the form $###,###,###
        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

            # remove dollar sign and commas
            s = re.sub('\$|,','', s)

            # convert to float
            value = float(s)

            # return value
            return value

        # otherwise, return NaN
        else:
            return np.nan
    
        
    # 13. Clean the box office column in the wiki_movies_df DataFrame.
    wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    #wiki_movies_df.drop('Box office', axis=1, inplace=True)
    # 14. Clean the budget column in the wiki_movies_df DataFrame.
    wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    #wiki_movies_df.drop('Budget', axis=1, inplace=True)
    # 15. Clean the release date column in the wiki_movies_df DataFrame.
    wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)
    #wiki_movies_df.drop('Release date', axis=1, inplace=True)
    # 16. Clean the running time column in the wiki_movies_df DataFrame.
    wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)
    #wiki_movies_df.drop('Running time', axis=1, inplace=True)
    # Return three variables. The first is the wiki_movies_df DataFrame
    
    return wiki_movies_df, kaggle_metadata, ratings 

In [64]:
# 17. Create the path to your file directory and variables for the three files.
file_dir = 'C:/Users/indra/Desktop/Class'
# The Wikipedia data
wiki_file = f'{file_dir}/wikipedia-movies.json'
# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# The MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [65]:
# 18. Set the three variables equal to the function created in D1.
wiki_file, kaggle_file, ratings_file = extract_transform_load(wiki_file, kaggle_file, ratings_file)

In [66]:
# 19. Set the wiki_movies_df equal to the wiki_file variable. 
wiki_movies_df = wiki_file

In [67]:
# 20. Check that the wiki_movies_df DataFrame looks like this. 
wiki_movies_df.head()

,Based on,Box office,Budget,Cinematography,Country,Directed by,Distributed by,Edited by,Language,Music by,...,Written by,imdb_link,title,url,year,imdb_id,box_office,budget,release_date,running_time
0,"[Characters, by Rex Weiner]",$21.4 million,$20 million,Oliver Wood,United States,Renny Harlin,20th Century Fox,Michael Tronick,English,"[Cliff Eidelman, Yello]",...,NaN,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,https://en.wikipedia.org/wiki/The_Adventures_o...,1990.0,tt0098987,21400000.0,20000000.0,1990-07-11,102.0
1,"[the novel, After Dark, My Sweet, by, Jim Thom...",$2.7 million,$6 million,Mark Plummer,United States,James Foley,Avenue Pictures,Howard E. Smith,English,Maurice Jarre,...,NaN,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990.0,tt0098994,2700000.0,6000000.0,1990-05-17,114.0
2,"[Air America, by, Christopher Robbins]","$57,718,089",$35 million,Roger Deakins,United States,Roger Spottiswoode,TriStar Pictures,"[John Bloom, Lois Freeman-Fox]","[English, Lao]",Charles Gross,...,NaN,https://www.imdb.com/title/tt0099005/,Air America,https://en.wikipedia.org/wiki/Air_America_(film),1990.0,tt0099005,57718089.0,35000000.0,1990-08-10,113.0
3,NaN,"$7,331,647",$12 million,Carlo Di Palma,United States,Woody Allen,Orion Pictures,Susan E. Morse,English,NaN,...,Woody Allen,https://www.imdb.com/title/tt0099012/,Alice,https://en.wikipedia.org/wiki/Alice_(1990_film),1990.0,tt0099012,7331647.0,12000000.0,1990-12-25,106.0
4,NaN,"$6,939,946 (USA)",$25 million,Russell Boyd,US,John Cornell,Paramount Pictures,David Stiven,English,Maurice Jarre,...,Paul Hogan,https://www.imdb.com/title/tt0099018/,Almost an Angel,https://en.wikipedia.org/wiki/Almost_an_Angel,1990.0,tt0099018,6939946.0,25000000.0,1990-12-19,95.0


In [62]:
# 21. Check that wiki_movies_df DataFrame columns are correct. 
wiki_movies_df.columns.to_list()

['Based on',
 'Cinematography',
 'Country',
 'Directed by',
 'Distributed by',
 'Edited by',
 'Language',
 'Music by',
 'Produced by',
 'Productioncompanies ',
 'Productioncompany ',
 'Screenplay by',
 'Starring',
 'Story by',
 'Written by',
 'imdb_link',
 'title',
 'url',
 'year',
 'imdb_id',
 'box_office',
 'budget',
 'release_date',
 'running_time']